In [1]:
from pathlib import Path

import matplotlib.pyplot as plt

from kedro.extras.datasets.pickle import PickleDataSet
from kedro.config import ConfigLoader

import kornia.augmentation as K

import torch

In [2]:
import os, sys
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src/'))

from src.tagseg.data.acdc_dataset import AcdcDataSet
from src.tagseg.data.dmd_dataset import DmdDataSet, DmdTimeDataSet
from src.tagseg.pipelines.data_processing.nodes import preprocess_acdc
from src.tagseg.pipelines.data_splitting.nodes import split_data

In [3]:
conf_paths = ["../conf/base", "../conf/local"]
conf_loader = ConfigLoader(conf_paths)
conf_catalog = conf_loader.get("catalog*", "catalog*/**")

In [4]:
dataset = DmdTimeDataSet(filepath='../' + conf_catalog['dmd_time_data']['filepath']).load()

In [7]:
torch.empty((4, 512, 16, 16)).flatten(start_dim=1, end_dim=3).shape

torch.Size([4, 131072])

In [25]:
proba = .2

train_aug = K.AugmentationSequential(
    K.RandomHorizontalFlip(p=proba),
    K.RandomVerticalFlip(p=proba),
    K.RandomElasticTransform(p=proba),
    K.RandomGaussianNoise(p=proba),
    K.RandomSharpness(p=proba),
    K.RandomGaussianBlur(kernel_size=(3, 3), sigma=(0.1, 0.1), p=proba),
    data_keys=["input", "mask"],
)

In [31]:
a, b = train_aug(data.tensors[0][:8], torch.empty_like(data.tensors[0][:8]))
a.shape, b.shape

(torch.Size([8, 1, 256, 256]), torch.Size([8, 1, 256, 256]))

In [20]:
videos = dataset.tensors[1]
torch.cat(videos[:, 1:].unbind()).unsqueeze(1).shape

torch.Size([1008, 1, 256, 256])

In [21]:
data = DmdDataSet(filepath='../' + conf_catalog['dmd_data']['filepath']).load()

In [22]:
data.tensors[0].shape

torch.Size([42, 1, 256, 256])

In [ ]:
tagged = AcdcDataSet(
            filepath='../' + conf_catalog["raw_acdc_data"]["filepath"],
            tagged=True,
            only_myo=True,
        ).load()

In [ ]:
cine = acdc = AcdcDataSet(
            filepath='../' + conf_catalog["raw_acdc_data"]["filepath"],
            tagged=False,
            only_myo=True,
        ).load()

In [ ]:
len(cine)

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(20, 8))

for i in range(13, 18):
    ax[0, i % 5].imshow(cine[4 * i + 21][0][0], cmap='gray');

    ax[0, i % 5].axis('off')
    
    ax[1, i % 5].imshow(tagged[4 * i + 21][0][0], cmap='gray')
    ax[1, i % 5].axis('off')

In [ ]:
fig, ax = plt.subplots(2, 10, figsize=(100, 20))

for col in range(10):

    image, label = dataset[col]

    ax[0, col].imshow(image[0], cmap='gray')
    ax[1, col].imshow(label)

In [ ]:
loaders = split_data(dataset, 0.75, 32)
loader_train, loader_val = loaders['loader_train'], loaders['loader_val']

In [ ]:
images, labels = next(iter(loader_train))
images.shape, labels.shape